In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [6]:
%autoreload
from pathlib import Path
import sys
root = Path().resolve().absolute().parent.parent
print(root)
sys.path.append(str(root))

from src.pipeline import Pipeline, BuilingIdsEnum
pipe = Pipeline()



/Users/matsalexander/Desktop/SolarEnergyImpact


In [7]:
building_b = pipe.get_data(BuilingIdsEnum.B)

# remove from 2024-01-15 from dataset A
mask = (building_b['timestamp'] >= '2024-01-15') & (building_b['timestamp'] <= '2024-01-15')
building_b = building_b[~mask]
# let value_import be equal to net_consumption for main


# normalize the value_import by area and call it value for main and building_b
building_b['value'] = building_b['value_import'] / building_b['area']

combined_df = pd.concat([ building_b])
# reset index
combined_df = combined_df.reset_index(drop=True)
# Perform the train-test split with stratification based on 'building_id'
train_data, test_data = train_test_split(
    combined_df,
    test_size=0.20,
    random_state=42,
    stratify=combined_df['building']
)

# select features
target = "value"
features = ["timestamp", "area", "temperature", "wind_speed"] #, "cloud_fraction", "precipitation"

train_data = train_data[features + [target]]
test_data = test_data[features + [target]]



In [8]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN']
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20241112_110853"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       13.45 GB / 32.00 GB (42.0%)
Disk Space Avail:   614.23 GB / 926.35 GB (66.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal 

[1000]	valid_set's l1: 0.00385133
[2000]	valid_set's l1: 0.00377962
[3000]	valid_set's l1: 0.00373246
[4000]	valid_set's l1: 0.00370201
[5000]	valid_set's l1: 0.00368092
[6000]	valid_set's l1: 0.00366336
[7000]	valid_set's l1: 0.00364597
[8000]	valid_set's l1: 0.00363508
[9000]	valid_set's l1: 0.00362633
[10000]	valid_set's l1: 0.00362712
[1000]	valid_set's l1: 0.00377976
[2000]	valid_set's l1: 0.00368283
[3000]	valid_set's l1: 0.00363217
[4000]	valid_set's l1: 0.00359904
[5000]	valid_set's l1: 0.00357535
[6000]	valid_set's l1: 0.00355974
[7000]	valid_set's l1: 0.00354194
[8000]	valid_set's l1: 0.00352911
[9000]	valid_set's l1: 0.003521
[10000]	valid_set's l1: 0.00351844
[1000]	valid_set's l1: 0.00412508
[2000]	valid_set's l1: 0.00397942
[3000]	valid_set's l1: 0.00390434
[4000]	valid_set's l1: 0.00386401
[5000]	valid_set's l1: 0.00383888
[6000]	valid_set's l1: 0.00382487
[7000]	valid_set's l1: 0.00381785
[8000]	valid_set's l1: 0.00381445
[9000]	valid_set's l1: 0.00380643
[10000]	valid_

	Ran out of time, early stopping on iteration 9228. Best iteration is:
	[8983]	valid_set's l1: 0.00370273


KeyboardInterrupt: 

In [ ]:
# AutogluonModels/ag-20241109_185606
# predictor = TabularPredictor.load("AutogluonModels/ag-20241109_185606")
# Evaluate on test data
performance = predictor.evaluate(test_data)
# best model: ag-20241022_161331

print("Evaluation Performance:")
performance
# reset index 
test_data = test_data.reset_index(drop=True)
# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions


Computing feature importance via permutation shuffling for 4 features using 4142 rows with 5 shuffle sets...


Evaluation Performance:


	6.79s	= Expected runtime (1.36s per shuffle set)
	2.09s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
             importance    stddev       p_value  n  p99_high   p99_low
timestamp      0.003790  0.000075  1.841552e-08  5  0.003945  0.003636
temperature    0.003285  0.000047  4.952707e-09  5  0.003381  0.003188
area           0.002735  0.000068  4.589780e-08  5  0.002876  0.002595
wind_speed     0.000635  0.000032  7.981253e-07  5  0.000702  0.000569


In [ ]:
performance

{'mean_absolute_error': -0.0033329320087102667,
 'root_mean_squared_error': -0.004850775099255721,
 'mean_squared_error': -2.353001906355935e-05,
 'r2': 0.7085213089279626,
 'pearsonr': 0.8420261238965251,
 'median_absolute_error': -0.00227640745880669}

In [ ]:
# model location => AutogluonModels/ag-20241016_095906
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

data_predict = main_building[features]
data_predict


,timestamp,area,temperature,wind_speed
0,2023-07-01 00:00:00,1199,13.6,1.6
1,2023-07-01 01:00:00,1199,13.2,2.0
2,2023-07-01 02:00:00,1199,12.3,1.6
3,2023-07-01 03:00:00,1199,11.9,0.6
4,2023-07-01 04:00:00,1199,11.9,0.2
...,...,...,...,...
10338,2024-09-03 18:00:00,1199,17.8,2.5
10339,2024-09-03 19:00:00,1199,17.8,1.8
10340,2024-09-03 20:00:00,1199,17.7,1.1
10341,2024-09-03 21:00:00,1199,18.0,3.4


In [ ]:
prediciton1 = predictor.predict(data_predict)

In [ ]:
# save predicitons as a csv in data folder from root.
import datetime
from pathlib import Path


prediciton1_df = pd.DataFrame(prediciton1)
date_time = datetime.datetime.now().strftime("%Y%m%d")
my_path = Path().resolve().parent.parent / 'data'/ "pred" / 'prediction_b.csv'
# create folder
my_path.parent.mkdir(parents=True, exist_ok=True)
prediciton1_df.to_csv(my_path, index=False)